# IN PROGRESS

In [ ]:
import pandas as pd
import geopandas as gpd
import pydeck as pdk
import matplotlib.pyplot as plt
import numpy as np
import folium
from census import Census
from us import states

# Get a US Census API Key [here](https://api.census.gov/data/key_signup.html) 

Copy-paste your API Key when prompted when running the cell below.

In [ ]:
import os
from getpass import getpass

# Try to read from env first, otherwise prompt you
CENSUS_API_KEY = os.getenv("CENSUS_API_KEY") or getpass("Enter your Census API key: ")

os.environ["CENSUS_API_KEY"] = CENSUS_API_KEY

print("Key loaded, length:", len(CENSUS_API_KEY), "characters")

**B08201_004E - B08210_006E**: Total households with 2+ vehicles

In [ ]:
veh_vars = [
    "B08201_004E",  # 2 vehicles
    "B08201_005E",  # 3 vehicles
    "B08201_006E",  # 4+ vehicles
    "multiple_vehicles", # Custom title for multiple vehicles
    "B08201_003E",  # 1 vehicle (probably not needed)
]

In [ ]:
c = Census(CENSUS_API_KEY, year=2022)

# List of Bay-Area county FIPS:
bay_fips = ["001","013","041","055","075","081","085","095","097"]

In [ ]:
def get_vehicles_by_tract(veh_list, veh_var):
    for county in bay_fips:
        veh_list += c.acs5.state_county_tract(
            (veh_var,"NAME"),
            states.CA.fips,
            county,
            Census.ALL
        )

    df = pd.DataFrame(veh_list)
    df["GEOID"] = df.state + df.county + df.tract
    df = df[["GEOID",veh_var]]

    return df

In [ ]:
veh1 = []
df_veh1 = get_vehicles_by_tract(veh1, veh_vars[-1])

veh2 = []
df_veh2 = get_vehicles_by_tract(veh2, veh_vars[0])

veh3 = []
df_veh3 = get_vehicles_by_tract(veh3, veh_vars[1])

veh4 = []
df_veh4 = get_vehicles_by_tract(veh4, veh_vars[2])

In [ ]:
# df_veh2.head()

In [ ]:
# df_veh3.head()

In [ ]:
# df_veh4.head()

In [ ]:
veh_dfs = [df_veh2, df_veh3, df_veh4] # not including 1-vehicle data

# concatenate and then group/sum
combined = pd.concat(veh_dfs, ignore_index=True)

veh_df = (
    combined
      .groupby("GEOID", as_index=False)
      .sum()
)

veh_df.head()

# Compute the sum across all columns except GEOID
veh_df["multiple_vehicles"] = veh_df.iloc[:, 1:].sum(axis=1)

# Create a new DataFrame with just GEOID + that sum
total_veh = veh_df[["GEOID", "multiple_vehicles"]].copy()

# 3. Reset the index
total_veh = total_veh.reset_index(drop=True)

print(veh_df.head())
total_veh.head()

In [ ]:
def plot_vehicles_by_tract(category):
    veh_df, veh_var, num_cars = category
    # Load Bay-Area tracts shapefile
    path = "/Users/dsong/Library/CloudStorage/OneDrive-UniversityofIllinois-Urbana/Research/UROP 2025 - UAM/Demand Analysis/TIGER Line 2022 Tract/tl_2022_06_tract.shp"
    tracts = gpd.read_file(path)[["GEOID","geometry"]]
    bay_tracts = tracts[tracts.GEOID.str[:5].isin({"06001","06013","06041",
                                                "06055","06075","06081",
                                                "06085","06095","06097"})]

    # Merge population → GeoDataFrame
    gdf = bay_tracts.merge(veh_df, on="GEOID", how="left").fillna(0)

    # Plot choropleth
    fig, ax = plt.subplots(1,1, figsize=(10,10))
    gdf.plot(
        column=veh_var,
        cmap="viridis",
        legend=True,
        legend_kwds={"label":"Total Households","fmt":"{:,.0f}"},
        linewidth=0.1,
        edgecolor="gray",
        ax=ax
    )
    ax.set_title(f"Bay Area Total Households with {num_cars} by Census Tract (ACS 5-Year)")
    ax.axis("off")
    plt.show()

In [ ]:
veh1_category = (df_veh1, veh_vars[-1], "1 Car")
veh2_category = (df_veh2, veh_vars[0], "2 Cars")
veh3_category = (df_veh3, veh_vars[1], "3 Cars")
veh4_category = (df_veh4, veh_vars[2], "4+ Cars ")
vehT_category = (total_veh, veh_vars[3], "Multiple Vehicles")

plot_vehicles_by_tract(vehT_category)